# Explicando Retrieval Augmented Generation

Neste exemplo, vamos carregar um PDF com prospecto de um fundo de investimento para utilizá-lo como base de conhecimento para tratamentos de dúvidas.

Utilizaremos os modelos da OpenAI para gerar embeddings e respostas.

Os dados vetorizados serão armazenados no Astra/Cassandra para busca baseada na similaridade de vetores (Vector Search).

## Hands-on: Setup / Configurando ambiente

Inicialmente, vamos instalar e configurar o ambiente para execução dos modelos

In [ ]:
pip install ragstack-ai --upgrade 

In [2]:
# Carregando variáveis de ambiente
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)

True

### Conectando ao Astra/Cassandra

Para utilizar o Cassandra como Vector DB, crie um banco de dados com suporte a vector search em astra.datastax.com, gere um token de conexão e identifique o seu Endpoint de acesso. Informe os valores na célula abaixo:

In [3]:
import os, json
from getpass import getpass

from astrapy.db import AstraDB
ASTRA_DB_API_ENDPOINT = input("ASTRA_DB_API_ENDPOINT = ")
ASTRA_DB_APPLICATION_TOKEN = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

ASTRA_DB_API_ENDPOINT = https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com
ASTRA_DB_APPLICATION_TOKEN = ········


Pronto!

A conexão com o Astra é muito simples e você já pode armazenar seus documentos.

### LangChain, Cassandra como Vector Store e OpenAI

In [4]:
# Importando o índice, separadores de texto, e auxiliares para processamendo dos documentos
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Hands-on: Embeddings

Vamos converter nossa base de documentos em embeddings. 

Se você não sabe o que são embeddings, assista aqui: https://www.youtube.com/watch?v=fvRyziDmvoA

Utilizaremos o LangChain para simplificar a leitura dos PDFs, geração de chunks e posterior utilização dos dados.

In [5]:
# Neste exemplo, vou utilizar os embeddings da OpenAI.
# Aqui, importamos o LLM e os Embeddings
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
llm = OpenAI(temperature=0.1)
embedding_generator = OpenAIEmbeddings()

In [6]:
#Gerando um embedding para uma pergunta do usuário
emb = OpenAIEmbeddings().embed_query("Qual o risco do fundo de investimento?")
print(f"Dimensões no embedding: {len(emb)}")
print(f"5 primeiras dimensões do embedding: {emb[:5]}")


Dimensões no embedding: 1536
5 primeiras dimensões do embedding: [0.01560478775073706, -0.026357666071101002, 0.016221111894567722, -0.028770506832908865, -0.031025989467398273]


### Dados utilizados no exercício

O arquivo utilizado neste exercício está disponível no url: https://conteudos.xpi.com.br/fundos-de-investimento/ip-participacoes-ipg-fic-fia-bdr-nivel-i/

Apesar de este conteúdo estar disponível na internet, ele NÃO FOI considerado no treinamento do GPT, por isso o modelo de linguagem não é capaz de responder perguntas sobre este fundo.

Baixe o arquivo e grave no diretório "pdf"

### Definindo o Index/Tabela do Cassandra que vai armazenar os dados

In [7]:
# O LangChain possui suporte ao Cassandra, então vamos utilizá-lo
# (vamos usar o Astra, que é o Cassandra como serviço)
# Criando o índice no Astra

from langchain.vectorstores import AstraDB
kb_table_name = 'vs_investment_kb'
keyspace = 'default_keyspace'
file = './pdf/Lamina_12082452000149_v46.pdf'

## Criando o index e carregando

O método IndexCreator executa a criação da tabela e vincula qual modelo de embeddings será utilizado no índice

In [13]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=AstraDB,
    embedding=embedding_generator,
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=30,
    ),
    vectorstore_kwargs={
        'collection_name': kb_table_name,
        'token': ASTRA_DB_APPLICATION_TOKEN,
        'api_endpoint': ASTRA_DB_API_ENDPOINT
    },
)

In [14]:
# Carregando os dados de um fundo de investimento específico
#loader = PyPDFDirectoryLoader('./pdf')
print(f"Loading file {file}")
loader = PyPDFLoader(file)
index = index_creator.from_loaders([loader])

Loading file ./pdf/Lamina_12082452000149_v46.pdf


In [ ]:
# Carregando os dados de um fundo de investimento específico
#loader = PyPDFDirectoryLoader('./funds')
#loader = PyPDFLoader("./funds/RealInvFIM0623.pdf")
#index = index_creator.from_loaders([loader])

### Se o seu índice já estiver carregado...

... você pode referenciá-lo desta maneira.

In [8]:
AstraVectorStore = AstraDB(
    collection_name= kb_table_name,
    token= ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint= ASTRA_DB_API_ENDPOINT,
    embedding=embedding_generator
)

index = VectorStoreIndexWrapper(
    vectorstore=AstraVectorStore
)

### Conferindo os dados gravados

Faremos uma query no Vector Store para conferir como os dados foram gravados

In [9]:
from astrapy.db import AstraDBCollection

collection = AstraDBCollection(
    collection_name=kb_table_name, 
    token= ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint= ASTRA_DB_API_ENDPOINT
)


In [11]:
data = collection.find_one({})
data

{'data': {'document': {'_id': '4dfe4564796044c29b099a2a08624930',
   'content': 'Variação % do \nIndice IBXSP-5,01% 3,32% 8,87% 3,59% 1,93% -3,07% -7,59% 3,51% -2,63% -3,11% 5,57% -0,05% 4,16%\nContribuição em relação ao  Indice  \nIBXSP 2 3\n3,77% 0,70% -1,44% -0,76% 1,42% 3,41% 2,30% 1,41% -0,79% 1,71% -0,27% -2,56% 10,06%\n  2 Para o cálculo foram consideradas todas as casas decimais disponíveis. \n3 Com relação ao campo  Contribuição em relação ao Indice IBXSP segue explicação para leitura do resultado:',
   '$vector': [-0.024794632932898467,
    0.0013354832968191772,
    0.01999697304811922,
    -0.02558752373166139,
    -0.023396995262012926,
    0.006171780728987985,
    -0.020668912982341477,
    -0.024875266842592243,
    -0.014917096342056933,
    -0.035639763401549084,
    0.012847516874303957,
    0.02132741579425995,
    -0.005768615837132043,
    -0.01043524794696537,
    -0.003255555453998824,
    0.04604141481746464,
    0.012719848379933988,
    0.02203967454597427,
 

# Hands-on:  Q/A - Utilizando o documento para responder uma pergunta

O LangChain possui métodos que buscam documentos similares no index e compõem um prompt para gerar a resposta a uma pergunta.

Este método são o ``ìndex.query`` e ``index.query_with_sources``.

A diferença entre estes métodos é o tipo de RetrievalChain que são utilizados por cada um.


In [12]:
import langchain
langchain.debug = False
langchain.verbose = True

QUERY = "Qual o rendimento do fundo?"

print("\nResposta com o query_with_sources:")
print(index.query_with_sources(question=QUERY))


Resposta com o query_with_sources:


> Entering new RetrievalQAWithSourcesChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not consti


> Finished chain.

> Finished chain.

> Finished chain.
{'question': 'Qual o rendimento do fundo?', 'answer': ' O rendimento do fundo nos últimos 5 anos foi de 56,89%, com uma variação do índice IBXSP de 54,15%. As despesas do fundo teriam custado R$ 22,16.\n', 'sources': './pdf/Lamina_12082452000149_v46.pdf'}


In [13]:
import langchain
langchain.debug = False
langchain.verbose = False

QUERY = "Qual o risco do fundo?"

print("Resposta com o query:")
print(index.query(question=QUERY))

Resposta com o query:
 O BNY Mellon Serviços Financeiros classifica os fundos que administra numa escala de 1 a 5 de acordo com o risco envolvido na estratégia de investimento de cada um deles. Nessa escala, a classificação do fundo é: 4,20.


Estes métodos funcionam bem, mas na vida real precisaremos de mais flexibilidade e detalhamento no comportamento do LLM, modificando o prompt para fornecer mais instruções, ou filtrando documentos que queremos que sejam considerados.

Por isso, o melhor é usarmos as mesmas Chains utilizadas acima, mas agora podendo especificar mais detalhes. 

Vamos ver como fazer isso.



# Hands-on: Q/A passo a passo

## Buscando documentos pela similaridade

A busca pelos documentos que vão preencher o contexto pode utilizar estratégias distintas.

Iniciaremos com a busca por similaridade de vetores.

In [14]:
AstraVectorStore = AstraDB(
    embedding=embedding_generator,
    collection_name=kb_table_name,
    token=ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
)

matchesSim = AstraVectorStore.search(QUERY, search_type='similarity', k=4)
for i, doc in enumerate(matchesSim):
    print(f'[{i:2}]: "{doc.page_content}"')

[ 0]: "os riscos do FUNDO, especialmente no sentido de que  rentabilidade passada  não é garantia  de resultados  futuros e que a aplicação em fundos  não 
contam com garantia do  ADMINISTRADOR, da GESTORA, de qualquer mecanismo de seguro ou do Fundo  Garantidor de Créditos – FGC. Os  
Distribuidores são orientados a somente utilizar materiais  de venda do FUNDO previamente avaliados pelo ADMINISTRADOR ou, quando aplicável, pela 
GESTORA."
[ 1]: "garantia junto às centrais depositárias, mas pode investir  em fundos de investimento que podem estar  expostos aos riscos  decorrentes de aplicações 
em ativos que incorram  em depósito de margem de garantia. As informações apresentadas são  provenientes dos fundos  investidos geridos por 
instituições ligadas.As estratégias de investimento do fundo podem resultar  em significativas perdas patrimoniais para  seus cotistas."
[ 2]: "Operações  compromissadas  lastreadas  em  títulos públicos federais
10,64%
Derivativos 0,86%
Cotas de fundos de 

## Utilizando a Chain com prompt personalizado
Vamos usar estes documentos para preencher o prompt. Note que nos próximos passos iremos importar as Chains e também definir o prompt que queremos utilizar.

In [15]:
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate

In [16]:
prompt_template = """
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["summaries", "question"]
)

In [17]:
langchain.verbose = True

retrieverSim = AstraVectorStore.as_retriever(
    search_type='similarity',
    search_kwargs={
        'k': 4,
        'filter': {"source": file}
    },
)

# Create a "RetrievalQA" chain
chainSim = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverSim,
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)
# Run it and print results
responseSim = chainSim.run(QUERY)
print(responseSim)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: Qual o risco do fundo?
os riscos do FUNDO, especialmente no sentido de que  rentabilidade passada  não é garantia  de resultados  futuros e que a aplicação em fundos  não 
contam com garantia do  ADMINISTRADOR, da GESTORA, de qualquer mecanismo de seguro ou do Fundo  Garantidor de Créditos – FGC. Os  
Distribuidores são orientados a somente utilizar materiais  de venda do FUNDO previamente avaliados pelo ADMINISTRADOR ou, quando aplicável, pela 
GESTORA.

garantia junto às centrais depositárias, mas pode investir  em fundos de investimento que podem est

In [16]:
# Busca com MMR (Maximal Marginal Relevance)
retrieverMMR = CassVectorStore.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 4,
        'fetch_k': 10,
    },
    return_source_documents=True
)

# Create a "RetrievalQA" chain
chainMMR = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverMMR,
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)
# Run it and print results
responseMMR = chainMMR.run(QUERY)
print(responseMMR)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: Qual o rendimento do fundo?
Assumindo que a última  taxa total de despesas divulgada se  mantenha constante e que  o fundo tenha rentabilidade bruta hipotética de 10% ao ano nos  
próximos 3 e 5 anos, o retorno após  as despesas terem sido descontadas, considerando a mesma aplicação inicial de R$ 1.000,00 (mil reais), é 
apresentado na  tabela abaixo:
SIMULAÇÃO DAS DESPESAS 2026 2028
Saldo bruto  acumulado  (hipotético - rentabilidade bruta anual de 10%) R$ 1.331,00 R$ 1.610,51

Operações  compromissadas  lastreadas  em  títulos públicos federais
10,64%

# Hands-on: Memória adicionando memória ao contexto

Após a primeira interação, as interações podem demandar que o contexto da conversação seja considerado nas próximas interações.

Para isso, é necessário registrarmos as mensagens também no banco de dados

In [18]:
from langchain.memory import AstraDBChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory
memory_table_name = 'vs_investment_memory'

In [21]:
def get_answer(conversation_id, q):
    prompt_template = """
    Given the following extracted parts of a long document and a question, create a final answer in a very short format. 
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    Answer in Portuguese.


    QUESTION: {question}
    =========
    {summaries}
    =========
    FINAL ANSWER:"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["summaries", "question"]
    )

    message_history = AstraDBChatMessageHistory(
        session_id=conversation_id,
        token=ASTRA_DB_APPLICATION_TOKEN,
        api_endpoint=ASTRA_DB_API_ENDPOINT,
        collection_name=memory_table_name
    )

    memory = ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=message_history,
        max_token_limit=50,
        buffer=""
    )

    retrieverSim = AstraVectorStore.as_retriever(
        search_type='similarity_score_threshold',
        search_kwargs={
            'k': 5,
            "score_threshold": .8
        },
    )
    
    # Create a "RetrievalQA" chain
    chainSim = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retrieverSim,
        memory=memory,
        chain_type_kwargs={
            'prompt': PROMPT,
            'document_variable_name': 'summaries'
        }
    )
    
    # Run it and print results
    answer = chainSim.run(q)

    return answer

langchain.debug = False
langchain.verbose = False

res = get_answer('my_conv2',QUERY)
print("="*20)
print(f"Query: {QUERY}")
print(res)

Query: Qual o risco do fundo?
 O fundo não conta com garantia do administrador, gestora ou FGC, e pode estar exposto a riscos decorrentes de aplicações em ativos que incorram em depósito de margem de garantia. A classificação de risco é 4,20.


In [20]:
from langchain.chains import ConversationalRetrievalChain

def get_answer(conversation_id, q):
    prompt_template = """
    Given the following extracted parts of a long document and a question, create a final answer in a very short format. 
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    Answer in Portuguese.


    QUESTION: {question}
    =========
    Chat History:
    ========
    {chat_history}
    ========
    {summaries}
    =========
    FINAL ANSWER:"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["summaries", "question","chat_history"]
    )

    message_history = AstraDBChatMessageHistory(
        session_id=conversation_id,
        session= cassio.config.resolve_session(),
        keyspace= cassio.config.resolve_keyspace(),
        ttl_seconds=3600,
        table_name=memory_table_name
    )

    memory = ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=message_history,
        max_token_limit=50,
        buffer="",
        return_messages=True,
        memory_key="chat_history"
    )

    retrieverSim = AstraVectorStore.as_retriever(
        search_type='similarity_score_threshold',
        search_kwargs={
            'k': 5,
            'filter': {"source": file},
            "score_threshold": .8
        },
    )

    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever = retrieverSim,
        memory=memory
    )

    answer = qa({"question": q})
    
    return answer

In [22]:
langchain.debug = False
langchain.verbose = True
res = get_answer('my_conv3',"Quanto dinheiro o fundo tem?")
print("="*20)
print(res)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Given the following extracted parts of a long document and a question, create a final answer in a very short format. 
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    Answer in Portuguese.


    QUESTION: Quanto dinheiro o fundo tem?
    nem solicitado resgates durante  o ano,  no primeiro dia útil de 2023, você poderia resgatar  R$ 749,34, já  deduzidos impostos no valor de R$  0,00.
DESPESAS:  As despesas do fundo, incluindo a taxa de administração,  a taxa de performance  e  as despesas operacionais e de serviços teriam custado 
R$ 22,16.

Ouvidoria: 0800  725 3219 ou ouvidoria@bn ymellon. com.br
3IP PARTICIPAÇÕES IPG FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE 
INVESTIMENTO EM AÇÕES  – BDR NÍVEL I
CNPJ: 12.082.452/0001-49Gestor: INVESTIDOR PROFISSIONAL GESTAO DE REC.LTDA
Administrador:

In [23]:
retrieverMMR = AstraVectorStore.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 4,
        'fetch_k': 10,
    },
    return_source_documents=True
)

# Create a "RetrievalQA" chain
chainMMR = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverMMR,
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)
# Run it and print results
responseMMR = chainMMR.run(QUERY)
print(responseMMR)
# Create a "RetrievalQA" chain
chainSim = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverSim,
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)
# Run it and print results
responseSim = chainSim.run(QUERY)
print(responseSim)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: Qual o risco do fundo?
os riscos do FUNDO, especialmente no sentido de que  rentabilidade passada  não é garantia  de resultados  futuros e que a aplicação em fundos  não 
contam com garantia do  ADMINISTRADOR, da GESTORA, de qualquer mecanismo de seguro ou do Fundo  Garantidor de Créditos – FGC. Os  
Distribuidores são orientados a somente utilizar materiais  de venda do FUNDO previamente avaliados pelo ADMINISTRADOR ou, quando aplicável, pela 
GESTORA.

Operações  compromissadas  lastreadas  em  títulos públicos federais
10,64%
Derivativos 0,86%
Cotas